In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [2]:
import tensorflow as tf
import pathlib
from tensorflow import keras
import glob
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [3]:
!unzip -q '/content/gdrive/MyDrive/Facial_Classification/images_260921.zip' -d /content/Data/

In [4]:
data_dir = pathlib.Path('/content/Data')

In [33]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                                data_dir,
                                                                validation_split = 0.2,
                                                                subset = 'training',
                                                                seed = 123,
                                                                image_size = (224,224),
                                                                batch_size = 32
                                                              )

Found 1773 files belonging to 3 classes.
Using 1419 files for training.


In [34]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                            data_dir,
                                                            validation_split = 0.2,
                                                            subset = 'validation',
                                                            seed = 123,
                                                            image_size = (224,224),
                                                            batch_size = 32
                                                            )

Found 1773 files belonging to 3 classes.
Using 354 files for validation.


In [26]:
IMG_SIZE = 224

In [27]:
base_model = keras.applications.MobileNetV2(weights='imagenet',
                                              input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                              include_top=False)

base_model.trainable = False

In [38]:
from keras.layers import  GlobalAveragePooling2D

In [39]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def model_maker():
  inputs = keras.Input(shape=(224,224,3))
  x = preprocess_input(inputs)
  x = base_model(x, training=False)
  x = GlobalAveragePooling2D()(x)
  x = Dense(128, activation = 'relu')(x)
  x = Dropout(0.25)(x)
  outputs = Dense(3, activation='softmax')(x)
  model = keras.Model(inputs, outputs)
  return model

In [40]:
model = model_maker()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv_7 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_7 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0   

In [41]:
model.compile(optimizer='adam',
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

In [42]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callback = [EarlyStopping(monitor = 'val_loss', 
                          patience=10, 
                          restore_best_weights=True),

            ReduceLROnPlateau(monitor = 'val_loss',
                              patience = 10,
                              factor = 0.7,
                              min_lr = 1e-2),

            ModelCheckpoint(filepath="/content/gdrive/MyDrive/Facial_Classification/model_1.h5",
                                                 save_weights_only=False, # the whole model (False) or only weights (True) 
                                                 save_best_only=True, # keep the best model with lowest validation loss
                                                 monitor='val_loss',
                                                 verbose=1 )]


In [43]:
history = model.fit(train_ds,
                              validation_data = val_ds,
                              epochs = 10,
                              callbacks = callback)

Epoch 1/10
45/45 [==============================] - 48s 1s/step - loss: 0.1247 - accuracy: 0.9535 - val_loss: 0.0368 - val_accuracy: 0.9859

Epoch 00001: val_loss improved from inf to 0.03677, saving model to /content/gdrive/MyDrive/Facial_Classification/model_1.h5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
45/45 [==============================] - 45s 995ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 0.0059 - val_accuracy: 0.9972

Epoch 00002: val_loss improved from 0.03677 to 0.00590, saving model to /content/gdrive/MyDrive/Facial_Classification/model_1.h5
Epoch 3/10
45/45 [==============================] - 45s 992ms/step - loss: 0.0117 - accuracy: 0.9965 - val_loss: 0.0083 - val_accuracy: 0.9972

Epoch 00003: val_loss did not improve from 0.00590
Epoch 4/10
45/45 [==============================] - 45s 987ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0064 - val_accuracy: 0.9972

Epoch 00004: val_loss did not improve from 0.00590
Epoch 5/10
45/45 [==============================] - 45s 991ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 0.9972

Epoch 00005: val_loss improved from 0.00590 to 0.00390, saving model to /content/gdrive/MyDrive/Facial_Classification/model_1.h5
Epoch 6/10
45/45 [==============================] - 45s 987ms/st